In [1]:
# librerias

import numpy as np
import pandas as pd
from pyomo.environ import *
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

import sys
import os

## Caso de transporte optimo

In [2]:
# Definir elementos

demand = {
   'Lon': 125,        # London
   'Ber': 175,        # Berlin
   'Maa': 225,        # Maastricht
   'Ams': 250,        # Amsterdam
   'Utr': 225,        # Utrecht
   'Hag': 200         # The Hague
}

supply = {
    'Arn': 600,        # Arnhem
   'Gou': 650         # Gouda
}


costos = {
    ('Lon', 'Arn'): 1000,
    ('Lon', 'Gou'): 2.5,
    ('Ber', 'Arn'): 2.5,
    ('Ber', 'Gou'): 1000,
    ('Maa', 'Arn'): 1.6,
    ('Maa', 'Gou'): 2.0,
    ('Ams', 'Arn'): 1.4,
    ('Ams', 'Gou'): 1.0,
    ('Utr', 'Arn'): 0.8,
    ('Utr', 'Gou'): 1.0,
    ('Hag', 'Arn'): 1.4,
    ('Hag', 'Gou'): 0.8
}

In [3]:
# Modelamiento y solcion

# Crear modelo
model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT)


# Paso 1 - Definir indices de conjuntos
CUS = list(demand.keys())
SRC = list(supply.keys())


# Paso 2 - Definir variables de decision
model.x = Var(CUS,SRC, domain= NonNegativeReals)


# Paso 3 - Definir funcion objetivo
@model.Objective(sense=minimize)
def cost(m):
    return sum([costos[c,s]*model.x[c,s] for c in CUS for s in SRC])


# Paso 4 - Restriccions
@model.Constraint(SRC)
def src(m,s):
    return sum([model.x[c,s] for c in CUS]) <= supply[s]

@model.Constraint(CUS)
def dmd(m,c):
    return sum([model.x[c,s] for s in SRC]) == demand[c]


# ver modelo
model.pprint()

5 Set Declarations
    dmd_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {'Lon', 'Ber', 'Maa', 'Ams', 'Utr', 'Hag'}
    src_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'Arn', 'Gou'}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   12 : {('Lon', 'Arn'), ('Lon', 'Gou'), ('Ber', 'Arn'), ('Ber', 'Gou'), ('Maa', 'Arn'), ('Maa', 'Gou'), ('Ams', 'Arn'), ('Ams', 'Gou'), ('Utr', 'Arn'), ('Utr', 'Gou'), ('Hag', 'Arn'), ('Hag', 'Gou')}
    x_index_0 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    6 : {'Lon', 'Ber', 'Maa', 'Ams', 'Utr', 'Hag'}
    x_index_1 : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
       

In [4]:
# Resolver problema de optimizacion
resultados = SolverFactory("glpk").solve(model)

# Resumen de optimizacion
resultados.write()

# Mostrar resultados
model.x.display()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1705.0
  Upper bound: 1705.0
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 13
  Number of nonzeros: 25
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.05044674873352051
# ----------------------------------------------------------
#   Solution Information
# ---------------------------------

In [5]:
# Ver valor de cada variable de decision 

for c in CUS:
    for s in SRC:
        print(c,s,model.x[c,s]())

Lon Arn 0.0
Lon Gou 125.0
Ber Arn 175.0
Ber Gou 0.0
Maa Arn 225.0
Maa Gou 0.0
Ams Arn 0.0
Ams Gou 250.0
Utr Arn 200.0
Utr Gou 25.0
Hag Arn 0.0
Hag Gou 200.0


In [6]:
# Mostrar resultados

if "ok" == str(resultados.Solver.status):
    print('Valor funcion objetivo:',model.cost())
    
    for s in SRC:
        for c in CUS:
            if model.x[c,s]():
                print('source:',s," - customer:",c," : ",model.x[c,s]())

else:
    print('No hay solucion al problema planteado')

Valor funcion objetivo: 1705.0
source: Arn  - customer: Ber  :  175.0
source: Arn  - customer: Maa  :  225.0
source: Arn  - customer: Utr  :  200.0
source: Gou  - customer: Lon  :  125.0
source: Gou  - customer: Ams  :  250.0
source: Gou  - customer: Utr  :  25.0
source: Gou  - customer: Hag  :  200.0


In [7]:
# Analisis de sensibilidad

#############################
# Source

print('Analisis de sensibilidad por Source')
if 'ok' == str(resultados.Solver.status):
    print("Source      Capacity   Shipped    Margin")
    for s in SRC:
        print(f"{s:10s}{supply[s]:10.1f}{model.src[s]():10.1f}{model.dual[model.src[s]]:10.4f}")
else:
    print("No hay solucion valida")

#############################
# Customer

print('\n\nAnalisis de sensibilidad por Customer')
if 'ok' == str(resultados.Solver.status):    
    print("Customer      Demand   Shipped    Margin")
    for c in CUS:
        print(f"{c:10s}{demand[c]:10.1f}{model.dmd[c]():10.1f}{model.dual[model.dmd[c]]:10.4f}")
else:
    print("No hay solucion valida")

Analisis de sensibilidad por Source
Source      Capacity   Shipped    Margin
Arn            600.0     600.0   -0.2000
Gou            650.0     600.0    0.0000


Analisis de sensibilidad por Customer
Customer      Demand   Shipped    Margin
Lon            125.0     125.0    2.5000
Ber            175.0     175.0    2.7000
Maa            225.0     225.0    1.8000
Ams            250.0     250.0    1.0000
Utr            225.0     225.0    1.0000
Hag            200.0     200.0    0.8000


## Caso ejemplo de seleccion de nadadores

In [39]:
# Definir valores 

valores =   [[43.5,None,48.4,38.2],
            [45.5,49.9,None,36.8],
            [43.4,None,42.1,43.2],
            [46.3,None,50.4,39.4],
            [48.7,53.6,49.1,None],
            [None,57.8,49.5,None],
            [47.9,None,45.7,39.8],
            [48.9,50.3,43.5,None]]

# Orden segun estudiante y estilos
students = ["Mario","Marcus","Luis","Lisandro","Pepe","Bruno","Ramiro","Christian"]
styles = ["backstroke","breastsroke","butterfly","freestyle"]


# Colocar en dataframe
datosT = pd.DataFrame(valores,index=students,columns=styles)
print(datosT)


# Todos los pares
costos = dict([
            ((students[i],styles[j]),valores[i][j]) 
            for i in range(len(students)) 
            for j in range(len(styles))
            if valores[i][j]   # solo aquellos que NO son None
            ])
# ver todos los pares
costos

           backstroke  breastsroke  butterfly  freestyle
Mario            43.5          NaN       48.4       38.2
Marcus           45.5         49.9        NaN       36.8
Luis             43.4          NaN       42.1       43.2
Lisandro         46.3          NaN       50.4       39.4
Pepe             48.7         53.6       49.1        NaN
Bruno             NaN         57.8       49.5        NaN
Ramiro           47.9          NaN       45.7       39.8
Christian        48.9         50.3       43.5        NaN


{('Mario', 'backstroke'): 43.5,
 ('Mario', 'butterfly'): 48.4,
 ('Mario', 'freestyle'): 38.2,
 ('Marcus', 'backstroke'): 45.5,
 ('Marcus', 'breastsroke'): 49.9,
 ('Marcus', 'freestyle'): 36.8,
 ('Luis', 'backstroke'): 43.4,
 ('Luis', 'butterfly'): 42.1,
 ('Luis', 'freestyle'): 43.2,
 ('Lisandro', 'backstroke'): 46.3,
 ('Lisandro', 'butterfly'): 50.4,
 ('Lisandro', 'freestyle'): 39.4,
 ('Pepe', 'backstroke'): 48.7,
 ('Pepe', 'breastsroke'): 53.6,
 ('Pepe', 'butterfly'): 49.1,
 ('Bruno', 'breastsroke'): 57.8,
 ('Bruno', 'butterfly'): 49.5,
 ('Ramiro', 'backstroke'): 47.9,
 ('Ramiro', 'butterfly'): 45.7,
 ('Ramiro', 'freestyle'): 39.8,
 ('Christian', 'backstroke'): 48.9,
 ('Christian', 'breastsroke'): 50.3,
 ('Christian', 'butterfly'): 43.5}

In [40]:
# Modelamiento y solucion (optimizacion)


# Crear modelo
model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT)


# Paso 1 - Definir indices de conjuntos
STUDENT = list(students)
STYLE = list(styles)

# Paso 2 - Definir variables de decision
model.x = Var(STUDENT,STYLE, domain = NonNegativeIntegers)


##################
### NOTA: La condicional final es para verificar que los valores existen en el diccionario
##################

# Paso 3 - Definir funcion objetivo - Tiempo minimo en los relevos (estilos)
@model.Objective(sense=minimize)
def cost(m):
    return sum([costos[e,s]*model.x[e,s] for e in STUDENT for s in STYLE if (e,s) in costos.keys()])

# Paso 4 - Restriccions
@model.Constraint(STUDENT)
def students(m,e):
    return sum([model.x[e,s] for s in STYLE if (e,s) in costos.keys()]) <= 1

@model.Constraint(STYLE)
def styles(m,s):
    return sum([model.x[e,s] for e in STUDENT if (e,s) in costos.keys()]) == 1

# ver modelo
model.pprint()

5 Set Declarations
    students_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    8 : {'Mario', 'Marcus', 'Luis', 'Lisandro', 'Pepe', 'Bruno', 'Ramiro', 'Christian'}
    styles_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'backstroke', 'breastsroke', 'butterfly', 'freestyle'}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   32 : {('Mario', 'backstroke'), ('Mario', 'breastsroke'), ('Mario', 'butterfly'), ('Mario', 'freestyle'), ('Marcus', 'backstroke'), ('Marcus', 'breastsroke'), ('Marcus', 'butterfly'), ('Marcus', 'freestyle'), ('Luis', 'backstroke'), ('Luis', 'breastsroke'), ('Luis', 'butterfly'), ('Luis', 'freestyle'), ('Lisandro', 'backstroke'), ('Lisandro', 'breastsroke'), ('Lisandro', 'butterfly'), ('Lisandro', 'fr

In [41]:
# Resolver problema de optimizacion
resultados = SolverFactory("glpk").solve(model)

# Resumen de optimizacion
resultados.write()

# Mostrar resultados
model.x.display()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 172.7
  Upper bound: 172.7
  Number of objectives: 1
  Number of constraints: 13
  Number of variables: 24
  Number of nonzeros: 47
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.04885363578796387
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------

In [42]:
# Mostrar resultados

if "ok" == str(resultados.Solver.status):
    print('Valor funcion objetivo:',model.cost())
else:
    print('No hay solucion al problema planteado')

Valor funcion objetivo: 172.7


In [43]:
# Mostrar resultados (Todo)

if "ok" == str(resultados.Solver.status):
    print('Valor funcion objetivo:',model.cost())
    
    for e in STUDENT:
        for s in STYLE:
            try:
                print(e,"-",s," - valor:",model.x[e,s]()," - tiempo:",costos[e,s])
            except:
                pass
else:
    print('No hay solucion al problema planteado')

Valor funcion objetivo: 172.7
Mario - backstroke  - valor: 1.0  - tiempo: 43.5
Mario - butterfly  - valor: 0.0  - tiempo: 48.4
Mario - freestyle  - valor: 0.0  - tiempo: 38.2
Marcus - backstroke  - valor: 0.0  - tiempo: 45.5
Marcus - breastsroke  - valor: 0.0  - tiempo: 49.9
Marcus - freestyle  - valor: 1.0  - tiempo: 36.8
Luis - backstroke  - valor: 0.0  - tiempo: 43.4
Luis - butterfly  - valor: 1.0  - tiempo: 42.1
Luis - freestyle  - valor: 0.0  - tiempo: 43.2
Lisandro - backstroke  - valor: 0.0  - tiempo: 46.3
Lisandro - butterfly  - valor: 0.0  - tiempo: 50.4
Lisandro - freestyle  - valor: 0.0  - tiempo: 39.4
Pepe - backstroke  - valor: 0.0  - tiempo: 48.7
Pepe - breastsroke  - valor: 0.0  - tiempo: 53.6
Pepe - butterfly  - valor: 0.0  - tiempo: 49.1
Bruno - breastsroke  - valor: 0.0  - tiempo: 57.8
Bruno - butterfly  - valor: 0.0  - tiempo: 49.5
Ramiro - backstroke  - valor: 0.0  - tiempo: 47.9
Ramiro - butterfly  - valor: 0.0  - tiempo: 45.7
Ramiro - freestyle  - valor: 0.0  - ti

In [44]:
# Mostrar resultados - Filtrados

if "ok" == str(resultados.Solver.status):
    print('Valor funcion objetivo:',model.cost())
    
    for e in STUDENT:
        for s in STYLE:
            try:
                if(model.x[e,s]() > 0):
                    print(e,"-",s," - valor:",model.x[e,s]()," - tiempo:",costos[e,s])
            except:
                pass
else:
    print('No hay solucion al problema planteado')

Valor funcion objetivo: 172.7
Mario - backstroke  - valor: 1.0  - tiempo: 43.5
Marcus - freestyle  - valor: 1.0  - tiempo: 36.8
Luis - butterfly  - valor: 1.0  - tiempo: 42.1
Christian - breastsroke  - valor: 1.0  - tiempo: 50.3


## Caso ejemplo de seleccion de nadadores 2

In [52]:
# Definir valores

times =  [[43.5,47.1,48.4,38.2],
            [45.5,42.1,49.6,36.8],
            [43.4,39.1,42.1,43.2],
            [46.5,44.1,44.5,41.2],
            [46.3,49.8,50.4,39.4],
            [48.7,49.6,49.1,35.2],
            [48.9,49.8,49.5,35.2],
            [44.1,48.8,50.4,37.2],
            [47.9,48.3,45.7,39.8],
            [48.9,46.3,43.5,38.6]]

# Orden segun estudiante y estilos
students = ["Mario","Marcus","Luis","Lisandro","Pepe","Bruno","Ramiro","Christian","Pablo","Sandro"]
styles = ["backstroke","breastsroke","butterfly","freestyle"]


# Colocar en dataframe
datosT = pd.DataFrame(times,index=students,columns=styles)
print(datosT)


# Todos los pares
costos = dict([
            ((students[i],styles[j]),times[i][j]) 
            for i in range(len(students)) 
            for j in range(len(styles))
            if times[i][j]
            ])
# ver todos los pares
costos

           backstroke  breastsroke  butterfly  freestyle
Mario            43.5         47.1       48.4       38.2
Marcus           45.5         42.1       49.6       36.8
Luis             43.4         39.1       42.1       43.2
Lisandro         46.5         44.1       44.5       41.2
Pepe             46.3         49.8       50.4       39.4
Bruno            48.7         49.6       49.1       35.2
Ramiro           48.9         49.8       49.5       35.2
Christian        44.1         48.8       50.4       37.2
Pablo            47.9         48.3       45.7       39.8
Sandro           48.9         46.3       43.5       38.6


{('Mario', 'backstroke'): 43.5,
 ('Mario', 'breastsroke'): 47.1,
 ('Mario', 'butterfly'): 48.4,
 ('Mario', 'freestyle'): 38.2,
 ('Marcus', 'backstroke'): 45.5,
 ('Marcus', 'breastsroke'): 42.1,
 ('Marcus', 'butterfly'): 49.6,
 ('Marcus', 'freestyle'): 36.8,
 ('Luis', 'backstroke'): 43.4,
 ('Luis', 'breastsroke'): 39.1,
 ('Luis', 'butterfly'): 42.1,
 ('Luis', 'freestyle'): 43.2,
 ('Lisandro', 'backstroke'): 46.5,
 ('Lisandro', 'breastsroke'): 44.1,
 ('Lisandro', 'butterfly'): 44.5,
 ('Lisandro', 'freestyle'): 41.2,
 ('Pepe', 'backstroke'): 46.3,
 ('Pepe', 'breastsroke'): 49.8,
 ('Pepe', 'butterfly'): 50.4,
 ('Pepe', 'freestyle'): 39.4,
 ('Bruno', 'backstroke'): 48.7,
 ('Bruno', 'breastsroke'): 49.6,
 ('Bruno', 'butterfly'): 49.1,
 ('Bruno', 'freestyle'): 35.2,
 ('Ramiro', 'backstroke'): 48.9,
 ('Ramiro', 'breastsroke'): 49.8,
 ('Ramiro', 'butterfly'): 49.5,
 ('Ramiro', 'freestyle'): 35.2,
 ('Christian', 'backstroke'): 44.1,
 ('Christian', 'breastsroke'): 48.8,
 ('Christian', 'butterfly'

In [53]:
# Modelamiento y solcion

# Crear modelo
model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT)


# Paso 1 - Definir indices de conjuntos
STUDENT = list(students)
STYLE = list(styles)


# Paso 2 - Definir variables de decision
#model.x = Var(STUDENT,STYLE, domain= NonNegativeIntegers)
model.x = Var(STUDENT,STYLE, domain= Binary)


##################
### NOTA: La condicional final es para verificar que los valores existen en el diccionario
##################

# Paso 3 - Definir funcion objetivo - Tiempo minimo en los relevos (estilos)
@model.Objective(sense=minimize)
def cost(m):
    return sum([costos[e,s]*model.x[e,s] for e in STUDENT for s in STYLE if (e,s) in costos.keys()])

# Paso 4 - Restriccions
@model.Constraint(STUDENT)
def students(m,e):
    return sum([model.x[e,s] for s in STYLE if (e,s) in costos.keys()]) <= 1

@model.Constraint(STYLE)
def styles(m,s):
    return sum([model.x[e,s] for e in STUDENT if (e,s) in costos.keys()]) == 1

# ver modelo 
model.pprint()

5 Set Declarations
    students_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {'Mario', 'Marcus', 'Luis', 'Lisandro', 'Pepe', 'Bruno', 'Ramiro', 'Christian', 'Pablo', 'Sandro'}
    styles_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'backstroke', 'breastsroke', 'butterfly', 'freestyle'}
    x_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   40 : {('Mario', 'backstroke'), ('Mario', 'breastsroke'), ('Mario', 'butterfly'), ('Mario', 'freestyle'), ('Marcus', 'backstroke'), ('Marcus', 'breastsroke'), ('Marcus', 'butterfly'), ('Marcus', 'freestyle'), ('Luis', 'backstroke'), ('Luis', 'breastsroke'), ('Luis', 'butterfly'), ('Luis', 'freestyle'), ('Lisandro', 'backstroke'), ('Lisandro', 'breastsroke'), ('Lisandro', 'butterfly'

In [54]:
# Resolver modelo planteado

# Resolver problema de optimizacion
resultados = SolverFactory("glpk").solve(model)

# Resumen de optimizacion
resultados.write()

# Mostrar resultados
model.x.display()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 161.3
  Upper bound: 161.3
  Number of objectives: 1
  Number of constraints: 15
  Number of variables: 41
  Number of nonzeros: 81
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.052722930908203125
# ----------------------------------------------------------
#   Solution Information
# ---------------------------------

In [55]:
# Mostrar resultados

if "ok" == str(resultados.Solver.status):
    print('Valor funcion objetivo:',model.cost())
else:
    print('No hay solucion al problema planteado')

Valor funcion objetivo: 161.3


In [56]:
# Mostrar resultados (Todo)

if "ok" == str(resultados.Solver.status):
    print('Valor funcion objetivo:',model.cost())
    
    for e in STUDENT:
        for s in STYLE:
            try:
                print(e,"-",s," - valor:",model.x[e,s]()," - tiempo:",costos[e,s])
            except:
                pass
else:
    print('No hay solucion al problema planteado')

Valor funcion objetivo: 161.3
Mario - backstroke  - valor: 1.0  - tiempo: 43.5
Mario - breastsroke  - valor: 0.0  - tiempo: 47.1
Mario - butterfly  - valor: 0.0  - tiempo: 48.4
Mario - freestyle  - valor: 0.0  - tiempo: 38.2
Marcus - backstroke  - valor: 0.0  - tiempo: 45.5
Marcus - breastsroke  - valor: 0.0  - tiempo: 42.1
Marcus - butterfly  - valor: 0.0  - tiempo: 49.6
Marcus - freestyle  - valor: 0.0  - tiempo: 36.8
Luis - backstroke  - valor: 0.0  - tiempo: 43.4
Luis - breastsroke  - valor: 1.0  - tiempo: 39.1
Luis - butterfly  - valor: 0.0  - tiempo: 42.1
Luis - freestyle  - valor: 0.0  - tiempo: 43.2
Lisandro - backstroke  - valor: 0.0  - tiempo: 46.5
Lisandro - breastsroke  - valor: 0.0  - tiempo: 44.1
Lisandro - butterfly  - valor: 0.0  - tiempo: 44.5
Lisandro - freestyle  - valor: 0.0  - tiempo: 41.2
Pepe - backstroke  - valor: 0.0  - tiempo: 46.3
Pepe - breastsroke  - valor: 0.0  - tiempo: 49.8
Pepe - butterfly  - valor: 0.0  - tiempo: 50.4
Pepe - freestyle  - valor: 0.0  - 

In [57]:
# Mostrar resultados - Filtrados

if "ok" == str(resultados.Solver.status):
    print('Valor funcion objetivo:',model.cost())
    
    for e in STUDENT:
        for s in STYLE:
            try:
                if(model.x[e,s]() > 0):
                    print(e,"-",s," - valor:",model.x[e,s]()," - tiempo:",costos[e,s])
            except:
                pass
else:
    print('No hay solucion al problema planteado')

Valor funcion objetivo: 161.3
Mario - backstroke  - valor: 1.0  - tiempo: 43.5
Luis - breastsroke  - valor: 1.0  - tiempo: 39.1
Ramiro - freestyle  - valor: 1.0  - tiempo: 35.2
Sandro - butterfly  - valor: 1.0  - tiempo: 43.5
